In [1]:
# LOAD PACKAGES
# -------------
using JuMP        # Need to say it whenever we use JuMP
using GLPK        # Loading the GLPK module for using its solver
using DataFrames  # Julia library for working with tabular data 
using CSV         # Julia library for handling delimited text files

# MODEL CONSTRUCTION
# ------------------
capstone = Model(GLPK.Optimizer) 
set_time_limit_sec(capstone::Model, 60) # We also set the time limit, in seconds

┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1278
┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1278


60

In [2]:
# INPUT DATA
# ----------

# Data on student preferences
Rank = DataFrame!(CSV.File("Rank.csv",header=false))
size(Rank)

# Data on Pillars
Pillar = DataFrame!(CSV.File("Pillar.csv",header=false))
size(Pillar)

# Data on Lower and Upper Bound (min and max number of students needed for each project from each discipline)
LowerBound = DataFrame!(CSV.File("LowerBound.csv",header=false))
UpperBound = DataFrame!(CSV.File("UpperBound.csv",header=false))
size(LowerBound)
size(UpperBound)

# Other data
Students = 1:170 # students
Projects = 1:61  # projects
PillarId = 1:4   # pillars

1:4

In [3]:
# VARIABLES
# ---------

# Binary variable taking value 1 if the s student is allocated to project p
@variable(capstone, x[Students,Projects], Bin)  

# Binary variable taking value 1 if project p is launched
@variable(capstone, y[Projects], Bin)          

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:61
And data, a 61-element Array{VariableRef,1}:
 y[1]
 y[2]
 y[3]
 y[4]
 y[5]
 y[6]
 y[7]
 y[8]
 y[9]
 y[10]
 y[11]
 y[12]
 y[13]
 ⋮
 y[50]
 y[51]
 y[52]
 y[53]
 y[54]
 y[55]
 y[56]
 y[57]
 y[58]
 y[59]
 y[60]
 y[61]

In [4]:
# CONSTRAINTS
# -----------

# Every student is allocated to a single project
@constraint(capstone, allocatestudent[i=Students], sum(x[i,j] for j = Projects)==1)

# A student is allocated to a project only if the project is launched
@constraint(capstone, allocateifoffered[i=Students,j=Projects],x[i,j]<=y[j])

# Number of student of the different discplines allocated to a project is within the min. and upp. bound
@constraint(capstone, lower[j=Projects,k=PillarId],sum(x[i,j]*Pillar[i,k] for i = Students) >= LowerBound[k,j]*y[j])
@constraint(capstone, upper[j=Projects,k=PillarId],sum(x[i,j]*Pillar[i,k] for i = Students) <= UpperBound[k,j]*y[j])

2-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},2,...} with index sets:
    Dimension 1, 1:61
    Dimension 2, 1:4
And data, a 61×4 Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},2}:
 upper[1,1] : x[1,1] + x[2,1] + x[3,1] + x[4,1] + x[5,1] + x[6,1] + x[7,1] + x[8,1] + x[9,1] + x[10,1] + x[11,1] + x[12,1] + x[13,1] + x[14,1] + x[15,1] + x[16,1] + x[17,1] + x[18,1] + x[19,1] + x[20,1] + x[21,1] + x[22,1] + x[23,1] + x[24,1] + x[25,1] + x[26,1] + x[27,1] + x[28,1] + x[29,1] + x[30,1] + x[31,1] + x[32,1] + x[33,1] + x[34,1] + x[35,1] + x[36,1] + x[37,1] + x[38,1] + x[39,1] + x[40,1] + x[41,1] + x[42,1] + x[43,1] + x[44,1] + x[45,1] + x[46,1] + x[47,1] + x[48,1] + x[49,1] + x[50,1] + x[51,1] + x[52,1] + x[53,1] + x[54,1] + x[55,1] + x[56,1] + x[57,1] + x[58

In [5]:
# OBJECTIVE
# ---------

# Maximize the total utility
@objective(capstone, Max, sum(Rank[i,j]*x[i,j] for i = Students, j = Projects))

# PRINT THE MODEL IN A HUMAN-READABLE FORMAT
# ------------------------------------------
println("The optimization problem to be solved is:")
print(capstone) # Shows the model constructed in a human-readable form

The optimization problem to be solved is:
Max 3 x[1,1] + 4 x[1,8] + x[1,9] + 2 x[1,14] + 8 x[1,20] + 5 x[1,23] + 6 x[1,31] + 9 x[1,37] + 7 x[1,42] + 10 x[1,54] + 7 x[2,1] + 5 x[2,7] + 6 x[2,12] + 3 x[2,14] + 8 x[2,20] + 9 x[2,22] + 4 x[2,25] + 10 x[2,37] + x[2,54] + 2 x[2,58] + x[3,2] + 5 x[3,7] + 7 x[3,8] + 8 x[3,9] + 4 x[3,14] + 3 x[3,30] + 6 x[3,31] + 10 x[3,37] + 9 x[3,54] + 2 x[3,58] + 4 x[4,4] + 10 x[4,7] + 5 x[4,8] + 6 x[4,9] + 9 x[4,10] + 2 x[4,23] + 3 x[4,37] + 8 x[4,40] + 7 x[4,46] + x[4,54] + 8 x[5,7] + 4 x[5,8] + 9 x[5,9] + 7 x[5,14] + 6 x[5,25] + 3 x[5,30] + 2 x[5,31] + 10 x[5,37] + x[5,54] + 5 x[5,58] + 8 x[6,7] + 9 x[6,8] + 10 x[6,9] + 5 x[6,21] + 3 x[6,30] + 4 x[6,33] + 7 x[6,37] + 6 x[6,40] + 2 x[6,46] + x[6,57] + 4 x[7,1] + 10 x[7,7] + 2 x[7,8] + 7 x[7,9] + 3 x[7,10] + 5 x[7,14] + 9 x[7,25] + x[7,31] + 6 x[7,46] + 8 x[7,58] + 10 x[8,1] + 8 x[8,7] + 7 x[8,8] + 9 x[8,12] + 3 x[8,21] + 2 x[8,22] + 4 x[8,23] + x[8,25] + 6 x[8,30] + 5 x[8,31] + 5 x[9,1] + 6 x[9,8] + 10 x[9

 allocatestudent[154] : x[154,1] + x[154,2] + x[154,3] + x[154,4] + x[154,5] + x[154,6] + x[154,7] + x[154,8] + x[154,9] + x[154,10] + x[154,11] + x[154,12] + x[154,13] + x[154,14] + x[154,15] + x[154,16] + x[154,17] + x[154,18] + x[154,19] + x[154,20] + x[154,21] + x[154,22] + x[154,23] + x[154,24] + x[154,25] + x[154,26] + x[154,27] + x[154,28] + x[154,29] + x[154,30] + x[154,31] + x[154,32] + x[154,33] + x[154,34] + x[154,35] + x[154,36] + x[154,37] + x[154,38] + x[154,39] + x[154,40] + x[154,41] + x[154,42] + x[154,43] + x[154,44] + x[154,45] + x[154,46] + x[154,47] + x[154,48] + x[154,49] + x[154,50] + x[154,51] + x[154,52] + x[154,53] + x[154,54] + x[154,55] + x[154,56] + x[154,57] + x[154,58] + x[154,59] + x[154,60] + x[154,61] = 1.0
 allocatestudent[155] : x[155,1] + x[155,2] + x[155,3] + x[155,4] + x[155,5] + x[155,6] + x[155,7] + x[155,8] + x[155,9] + x[155,10] + x[155,11] + x[155,12] + x[155,13] + x[155,14] + x[155,15] + x[155,16] + x[155,17] + x[155,18] + x[155,19] + x[155,

 lower[20,2] : x[60,20] + x[61,20] + x[62,20] + x[63,20] + x[64,20] + x[65,20] + x[66,20] + x[67,20] + x[68,20] + x[69,20] + x[70,20] + x[71,20] + x[72,20] + x[73,20] + x[74,20] + x[75,20] + x[76,20] + x[77,20] + x[78,20] + x[79,20] + x[80,20] + x[81,20] + x[82,20] + x[83,20] + x[84,20] + x[85,20] + x[86,20] + x[87,20] + x[88,20] + x[89,20] + x[90,20] + x[91,20] + x[92,20] + x[93,20] + x[94,20] + x[95,20] + x[96,20] + x[97,20] + x[98,20] + x[99,20] + x[100,20] + x[101,20] + x[102,20] + x[103,20] + x[104,20] + x[105,20] + x[106,20] + x[107,20] + x[108,20] + x[109,20] + x[110,20] + x[111,20] + x[112,20] + x[113,20] + x[114,20] + x[115,20] + x[116,20] + x[117,20] + x[118,20] + x[119,20] + x[120,20] + x[121,20] - y[20] ≥ 0.0
 lower[21,2] : x[60,21] + x[61,21] + x[62,21] + x[63,21] + x[64,21] + x[65,21] + x[66,21] + x[67,21] + x[68,21] + x[69,21] + x[70,21] + x[71,21] + x[72,21] + x[73,21] + x[74,21] + x[75,21] + x[76,21] + x[77,21] + x[78,21] + x[79,21] + x[80,21] + x[81,21] + x[82,21] + x

 lower[48,4] : x[150,48] + x[151,48] + x[152,48] + x[153,48] + x[154,48] + x[155,48] + x[156,48] + x[157,48] + x[158,48] + x[159,48] + x[160,48] + x[161,48] + x[162,48] + x[163,48] + x[164,48] + x[165,48] + x[166,48] + x[167,48] + x[168,48] + x[169,48] + x[170,48] ≥ 0.0
 lower[49,4] : x[150,49] + x[151,49] + x[152,49] + x[153,49] + x[154,49] + x[155,49] + x[156,49] + x[157,49] + x[158,49] + x[159,49] + x[160,49] + x[161,49] + x[162,49] + x[163,49] + x[164,49] + x[165,49] + x[166,49] + x[167,49] + x[168,49] + x[169,49] + x[170,49] ≥ 0.0
 lower[50,4] : x[150,50] + x[151,50] + x[152,50] + x[153,50] + x[154,50] + x[155,50] + x[156,50] + x[157,50] + x[158,50] + x[159,50] + x[160,50] + x[161,50] + x[162,50] + x[163,50] + x[164,50] + x[165,50] + x[166,50] + x[167,50] + x[168,50] + x[169,50] + x[170,50] ≥ 0.0
 lower[51,4] : x[150,51] + x[151,51] + x[152,51] + x[153,51] + x[154,51] + x[155,51] + x[156,51] + x[157,51] + x[158,51] + x[159,51] + x[160,51] + x[161,51] + x[162,51] + x[163,51] + x[16

 allocateifoffered[28,8] : x[28,8] - y[8] ≤ 0.0
 allocateifoffered[29,8] : x[29,8] - y[8] ≤ 0.0
 allocateifoffered[30,8] : x[30,8] - y[8] ≤ 0.0
 allocateifoffered[31,8] : x[31,8] - y[8] ≤ 0.0
 allocateifoffered[32,8] : x[32,8] - y[8] ≤ 0.0
 allocateifoffered[33,8] : x[33,8] - y[8] ≤ 0.0
 allocateifoffered[34,8] : x[34,8] - y[8] ≤ 0.0
 allocateifoffered[35,8] : x[35,8] - y[8] ≤ 0.0
 allocateifoffered[36,8] : x[36,8] - y[8] ≤ 0.0
 allocateifoffered[37,8] : x[37,8] - y[8] ≤ 0.0
 allocateifoffered[38,8] : x[38,8] - y[8] ≤ 0.0
 allocateifoffered[39,8] : x[39,8] - y[8] ≤ 0.0
 allocateifoffered[40,8] : x[40,8] - y[8] ≤ 0.0
 allocateifoffered[41,8] : x[41,8] - y[8] ≤ 0.0
 allocateifoffered[42,8] : x[42,8] - y[8] ≤ 0.0
 allocateifoffered[43,8] : x[43,8] - y[8] ≤ 0.0
 allocateifoffered[44,8] : x[44,8] - y[8] ≤ 0.0
 allocateifoffered[45,8] : x[45,8] - y[8] ≤ 0.0
 allocateifoffered[46,8] : x[46,8] - y[8] ≤ 0.0
 allocateifoffered[47,8] : x[47,8] - y[8] ≤ 0.0
 allocateifoffered[48,8] : x[48,8] - y[8

 allocateifoffered[76,15] : x[76,15] - y[15] ≤ 0.0
 allocateifoffered[77,15] : x[77,15] - y[15] ≤ 0.0
 allocateifoffered[78,15] : x[78,15] - y[15] ≤ 0.0
 allocateifoffered[79,15] : x[79,15] - y[15] ≤ 0.0
 allocateifoffered[80,15] : x[80,15] - y[15] ≤ 0.0
 allocateifoffered[81,15] : x[81,15] - y[15] ≤ 0.0
 allocateifoffered[82,15] : x[82,15] - y[15] ≤ 0.0
 allocateifoffered[83,15] : x[83,15] - y[15] ≤ 0.0
 allocateifoffered[84,15] : x[84,15] - y[15] ≤ 0.0
 allocateifoffered[85,15] : x[85,15] - y[15] ≤ 0.0
 allocateifoffered[86,15] : x[86,15] - y[15] ≤ 0.0
 allocateifoffered[87,15] : x[87,15] - y[15] ≤ 0.0
 allocateifoffered[88,15] : x[88,15] - y[15] ≤ 0.0
 allocateifoffered[89,15] : x[89,15] - y[15] ≤ 0.0
 allocateifoffered[90,15] : x[90,15] - y[15] ≤ 0.0
 allocateifoffered[91,15] : x[91,15] - y[15] ≤ 0.0
 allocateifoffered[92,15] : x[92,15] - y[15] ≤ 0.0
 allocateifoffered[93,15] : x[93,15] - y[15] ≤ 0.0
 allocateifoffered[94,15] : x[94,15] - y[15] ≤ 0.0
 allocateifoffered[95,15] : x[9

Excessive output truncated after 524330 bytes.

In [6]:
# SOLVE IT
# --------

status = optimize!(capstone)
termination_status(capstone)

TIME_LIMIT::TerminationStatusCode = 12

In [7]:
# DISPLAY THE RESULTS
# -------------------

println("Objective value: ", JuMP.objective_value(capstone)) 
# println("x = ", JuMP.value.(x), " y = ", JuMP.value.(y))
println("The average utility is: ", JuMP.objective_value(capstone)/maximum(Students)) 
println("The total number of projects is: ", sum(JuMP.value.(y))) 

Objective value: 1448.0
The average utility is: 8.51764705882353
The total number of projects is: 30.0
